In [6]:
from tsfresh.examples import load_robot_execution_failures
import tsfresh
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_relevant_features
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy import spatial
from datetime import timedelta
import seaborn as sns
sns.set_style("whitegrid")
from tsfresh import extract_features

df = pd.read_csv('D:\data from MnM Samdal raw\Samdal_power_1min_no_outliers.csv',  parse_dates=[0])


In [ ]:
df.precipitation = df.precipitation.fillna(value=0)
df.power = df.power.interpolate(method='linear', limit = 1 )
df.speed = df.speed.interpolate(method='linear', limit = 1 )

In [ ]:
df.describe()
df.corr()
df.cov()

In [7]:
df = df.rename( columns={"Unnamed: 0": "time"})
df = df.rename( columns={"speed": "wind_speed"})

# discard April 
df = df.drop(df[df.time.dt.month==4].index)

df['DayOfWeekNum'] = df['time'].dt.dayofweek
df['DayOfWeek'] = df['time'].dt.weekday_name
df['MonthDayNum'] = df['time'].dt.day
df['HourOfDay'] = df['time'].dt.hour
df['Month'] = df['time'].dt.month

In [ ]:
df = df.dropna()

In [4]:
# power curve scatterplot with histograms  

del df['time']

sns.set(font_scale=5)
sns.set(style="white", color_codes=True)
sns.set_context("poster",font_scale=1.5)

g = sns.jointplot(x="wind_speed", y="power", data=df)
g.set_axis_labels("speed [m/s]", "power [kW]",  fontsize=26)

plt.show()

In [10]:
# power curve heatmap with histograms  

# remove small values of power

df = df.drop(df[df.power<20].index)

# Spread , Shape
# variance , std dev
#skewness , kurtosis
sns.set(font_scale=5)
sns.set(style="white", color_codes=True)
sns.set_context("poster",font_scale=1.5)

g = sns.jointplot(x="wind_speed", y="power", data=df, kind="hex")
g.set_axis_labels("speed [m/s]", "power [kW]",  fontsize=26)

plt.show()

In [4]:
#monthly each month

meanpointprops = dict(marker='D', markeredgecolor='black',   markerfacecolor='green')

df.boxplot(column = ['wind_speed', 'wind_speed_10m'], by = 'Month', showmeans=True, meanprops=meanpointprops, fontsize=18)

plt.ylabel('wind speed [m/s]', fontsize=26)
plt.yticks(fontsize=26)
labels_len = [1, 2, 3, 4, 5,6]
labels = ['May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct']
plt.xticks(labels_len, labels)

df.boxplot(  column = ['power'], by = 'Month',showmeans=True, meanprops=meanpointprops, fontsize=26)
plt.ylabel('power [kW]', fontsize=26)
plt.yticks(fontsize=26)

labels_len = [1, 2, 3, 4, 5,6]
labels = ['May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct']
plt.xticks(labels_len, labels)
plt.xlabel('month', fontsize=26)

plt.show()

KeyError: 'Column not found: wind_speed_10m'

In [9]:
# Daily
# Central tendency: Mean, Median, Mode
# Dispersion: Range , Quantiles of the data-set

df.boxplot(column = ['wind_speed', 'wind_speed_10m'], by = 'HourOfDay',showmeans=True, meanprops=meanpointprops, fontsize=20)
plt.ylabel('wind speed [m/s]', fontsize=26)
plt.yticks(fontsize=26)
plt.xlabel('hour', fontsize=26)

df.boxplot(  column = ['power'], by = 'HourOfDay', showmeans=True, meanprops=meanpointprops,fontsize=26)
plt.ylabel('power [kW]', fontsize=26)
plt.yticks(fontsize=26)
plt.xlabel('hour', fontsize=26)

plt.show()

KeyError: 'Column not found: wind_speed_10m'

In [4]:
# multiple power curve plots

fig, axarr = plt.subplots(2, 3, sharex=True, sharey=True)

axarr[0, 0].scatter(df.wind_speed.loc[df.time.dt.month==5], df.power.loc[df.time.dt.month==5], alpha=0.7, c='g')     #c='cyan',
axarr[0, 0].set_title('May', fontsize=14)
axarr[0, 0].set_ylabel('power [kW]', fontsize=26)
axarr[0, 0].set_xlim(-0.5,20)
axarr[0, 0].set_ylim(-150,3000)

axarr[0, 1].scatter(df.wind_speed.loc[df.time.dt.month==6], df.power.loc[df.time.dt.month==6], alpha=0.7, c='orangered') #c='teal',
axarr[0, 1].set_title('June', fontsize=14)

axarr[0, 2].scatter(df.wind_speed.loc[df.time.dt.month==7], df.power.loc[df.time.dt.month==7], alpha=0.7, c='orangered') #c='teal',
axarr[0, 2].set_title('July', fontsize=14)

axarr[1, 0].scatter(df.wind_speed.loc[df.time.dt.month==8], df.power.loc[df.time.dt.month==8], alpha=0.7, c='orangered')     #c='darkviolet',
axarr[1, 0].set_title('August', fontsize=14)

axarr[1, 1].scatter(df.wind_speed.loc[df.time.dt.month==9], df.power.loc[df.time.dt.month==9], alpha=0.7) #c='salmon',
axarr[1, 1].set_title('September', fontsize=14)

axarr[1, 2].scatter(df.wind_speed.loc[df.time.dt.month==10], df.power.loc[df.time.dt.month==10], alpha=0.7) #c='salmon',
axarr[1, 2].set_title('October', fontsize=14)

axarr[1, 1].set_xlabel('wind speed [m/s]', fontsize=26)

fig.suptitle('Monthly power curves, [1s resolution] ', fontsize=28)

plt.show()


In [5]:
# multiple hist plots; wind speed

fig, axarr = plt.subplots(2, 3, sharex=True, sharey=True)

axarr[0, 0].hist(df.wind_speed.loc[df.time.dt.month==5].dropna(), alpha=0.8, color='g')      #c='cyan',
axarr[0, 0].set_title('May', fontsize=14)

axarr[0, 1].hist(df.wind_speed.loc[df.time.dt.month==6].dropna(), alpha=0.8, color='orangered') 
axarr[0, 1].set_title('June', fontsize=14)

axarr[0, 2].hist(df.wind_speed.loc[df.time.dt.month==7].dropna(), alpha=0.8, color='orangered') 
axarr[0, 2].set_title('July', fontsize=14)

axarr[1, 0].hist(df.wind_speed.loc[df.time.dt.month==8].dropna(), alpha=0.8, color='orangered')    
axarr[1, 0].set_title('August', fontsize=14)

axarr[1, 1].hist(df.wind_speed.loc[df.time.dt.month==9].dropna(), alpha=0.8, color='b') 
axarr[1, 1].set_title('September', fontsize=14)

axarr[1, 2].hist(df.wind_speed.loc[df.time.dt.month==10].dropna(), alpha=0.8, color='b') 
axarr[1, 2].set_title('October', fontsize=14)

axarr[1, 1].set_xlabel('wind speed [m/s]', fontsize=26)

fig.suptitle('Histograms of wind speed, [1s interval] ', fontsize=28)

plt.show()


In [17]:
# multiple hist plots; power

fig, axarr = plt.subplots(2, 3, sharex=True, sharey=True)

axarr[0, 0].hist(df.power.loc[df.time.dt.month==5].dropna(), alpha=0.8, color='g')     
axarr[0, 0].set_title('May', fontsize=14)


axarr[0, 1].hist(df.power.loc[df.time.dt.month==6].dropna(), alpha=0.8, color='orangered') 
axarr[0, 1].set_title('June', fontsize=14)

axarr[0, 2].hist(df.power.loc[df.time.dt.month==7].dropna(), alpha=0.8, color='orangered') 
axarr[0, 2].set_title('July', fontsize=14)

axarr[1, 0].hist(df.power.loc[df.time.dt.month==8].dropna(), alpha=0.8, color='orangered')    
axarr[1, 0].set_title('August', fontsize=14)

axarr[1, 1].hist(df.power.loc[df.time.dt.month==9].dropna(), alpha=0.8, color='b') 
axarr[1, 1].set_title('September', fontsize=14)

axarr[1, 2].hist(df.power.loc[df.time.dt.month==10].dropna(), alpha=0.8, color='b') 
axarr[1, 2].set_title('October', fontsize=14)

axarr[1, 1].set_xlabel('power [kW]', fontsize=26)

fig.suptitle('Histograms of power, [1s interval] ', fontsize=28)

plt.show()


In [ ]:
from scipy import stats

stats.describe(df.speed.dropna())

In [37]:
grouped_data = df.groupby(df.time.dt.month)
print(grouped_data)

grouped_data['power'].describe().unstack()     
#the method unstack is used to get the mean, standard deviation (std), etc as columns and it becomes somewhat easier to read.


,count,mean,std,min,25%,50%,75%,max
time,,,,,,,,
5,320.0,397.187821,567.118357,14.714593,14.908340,212.415170,487.374262,2507.615923
6,175.0,234.726774,299.650650,14.873653,14.904156,96.805396,355.798798,1412.220046
7,220.0,273.581696,411.310638,14.945439,15.164090,152.326165,323.275427,2501.700632
8,304.0,239.356441,250.007837,15.015097,15.139725,167.196834,346.151543,1063.117775
9,115.0,268.649461,321.155424,14.775001,14.983316,185.403046,367.364740,2496.619294
10,73.0,187.734968,213.005815,14.643242,14.681245,14.757341,330.796883,755.805750
